In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
bitcoin_tweets_df = pd.read_csv('Data/tweets/bitcoin_tweets_cleaned_sent.csv', encoding='ascii', parse_dates=True)

c:\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
bitcoin_tweets_df['date'] = pd.to_datetime(bitcoin_tweets_df['date'])

In [4]:
bitcoin_tweets_df = bitcoin_tweets_df.drop(["text","is_retweet", "sentiment"],axis =1)

In [5]:
bitcoin_tweets_df.set_index(bitcoin_tweets_df["date"],inplace=True)
bitcoin_tweets_df = bitcoin_tweets_df.drop(["date"],axis =1)

In [6]:
var_list=['subjectivity', 'polarity', 'vader_compound', 'vader_pos', 'vader_neg','vader_neu', 'afinn_score']

In [7]:
bitcoin_tweets_df_hour = bitcoin_tweets_df.resample('H').mean()
bitcoin_tweets_df_day = bitcoin_tweets_df.resample('D').mean()

In [8]:
bitcoin_tweets_df_day_cleaned=bitcoin_tweets_df_day.fillna(bitcoin_tweets_df_day.rolling(7,min_periods=1).mean())
bitcoin_tweets_df_day_cleaned[var_list] = bitcoin_tweets_df_day_cleaned[var_list].fillna(value=bitcoin_tweets_df_day_cleaned[var_list].mean())
bitcoin_tweets_df_hour_cleaned=bitcoin_tweets_df_hour.fillna(bitcoin_tweets_df_hour.rolling(168,min_periods=1).mean())
bitcoin_tweets_df_hour_cleaned[var_list] = bitcoin_tweets_df_hour_cleaned[var_list].fillna(value=bitcoin_tweets_df_hour_cleaned[var_list].mean())

In [9]:
def effectivesentiment(x):
    if x > 0:
        y = 1
    elif x < 0:
        y = -1
    else:
        y = 0
    return y

In [10]:
def agg_sentiment(x):
    
    y = 0
    
    if x[0] + x[1] + x[2] >0:
        y=1
    
    elif x[0] + x[1] + x[2] <0:
        y=-1
    
    else:
        y=0
    
    return y

In [11]:
def applyfunctions(df):
    x = df["vader_pos"] > df["vader_neg"]# filtering only posive rows neglecting negative and neutral ones
    y = df["vader_pos"] < df["vader_neg"]# filtering only negative rows neglecting positive and neutral ones

    df["effective_polarity"] = df["polarity"].apply(effectivesentiment)
    df["effective_afinn"] = df["polarity"].apply(effectivesentiment)
    df["effective_vader"] = 0 #intialize all with zeros and neutral tweets gets automatically zero
    df.loc[x,"effective_vader"] = 1
    df.loc[y,"effective_vader"] = -1
    df['agg'] = df[['effective_afinn',"effective_polarity", "effective_vader"]].values.tolist()
    df["agg_sentiment_polarity"] = df['agg'].apply(agg_sentiment)
    df = df.drop(["agg"],axis =1)
    return df

In [12]:
bitcoin_tweets_df, bitcoin_tweets_df_hour_cleaned, bitcoin_tweets_df_day_cleaned = applyfunctions(bitcoin_tweets_df), applyfunctions(bitcoin_tweets_df_hour_cleaned),applyfunctions(bitcoin_tweets_df_day_cleaned)

In [14]:
bitcoin_tweets_df_hour_agg = bitcoin_tweets_df.resample('H').mean()
bitcoin_tweets_df_day_agg = bitcoin_tweets_df.resample('D').mean()

In [15]:
var_list=['subjectivity', 'polarity', 'vader_compound', 'vader_pos', 'vader_neg',
       'vader_neu', 'afinn_score', 'effective_polarity', 'effective_afinn',
       'effective_vader', 'agg_sentiment_polarity']

In [17]:
bitcoin_tweets_df_day_agg_cleaned=bitcoin_tweets_df_day_agg.fillna(bitcoin_tweets_df_day_agg.rolling(7,min_periods=1).mean())
bitcoin_tweets_df_day_agg_cleaned[var_list] = bitcoin_tweets_df_day_agg_cleaned[var_list].fillna(value=bitcoin_tweets_df_day_agg_cleaned[var_list].mean())
bitcoin_tweets_df_hour_agg_cleaned=bitcoin_tweets_df_hour_agg.fillna(bitcoin_tweets_df_hour_agg.rolling(168,min_periods=1).mean())
bitcoin_tweets_df_hour_agg_cleaned[var_list] = bitcoin_tweets_df_hour_agg_cleaned[var_list].fillna(value=bitcoin_tweets_df_hour_agg_cleaned[var_list].mean())

In [18]:
btc_price_data = pd.read_csv('Data/prices/btcusd.csv')

In [19]:
btc_price_data['time'] = pd.to_datetime(btc_price_data['time'], unit='ms')
btc_price_data['date'] = btc_price_data['time'].dt.date
btc_price_data['hour'] = pd.to_datetime(btc_price_data['time']).dt.floor('H')
btc_price_data.head()

,time,open,close,high,low,volume,date,hour
0,2013-04-01 00:07:00,93.25,93.30,93.30,93.25,93.300000,2013-04-01,2013-04-01
1,2013-04-01 00:08:00,100.00,100.00,100.00,100.00,93.300000,2013-04-01,2013-04-01
2,2013-04-01 00:09:00,93.30,93.30,93.30,93.30,33.676862,2013-04-01,2013-04-01
3,2013-04-01 00:11:00,93.35,93.47,93.47,93.35,20.000000,2013-04-01,2013-04-01
4,2013-04-01 00:12:00,93.47,93.47,93.47,93.47,2.021627,2013-04-01,2013-04-01


In [20]:
btc_daily_open_close = btc_price_data[['time','date']].groupby(['date']).time.agg([min,max]).reset_index()
btc_daily_high=btc_price_data.groupby(['date']).high.agg(max).reset_index()
btc_daily_low=btc_price_data.groupby(['date']).low.agg(min).reset_index()
btc_daily_open_close.columns=['date','open_time','close_time']
btc_daily_open = pd.merge(btc_daily_open_close, btc_price_data, left_on='open_time', right_on='time')[['date_x','open','volume']]
btc_daily_close = pd.merge(btc_daily_open_close, btc_price_data, left_on='close_time', right_on='time')[['date_x','close','volume']]
btc_daily_high_low = pd.merge(btc_daily_high, btc_daily_low, on='date')
btc_daily_open_close = pd.merge(btc_daily_open, btc_daily_close, on='date_x')
btc_daily_open_close.columns=['date','open','open_volume','close','close_volume']
btc_daily=pd.merge(btc_daily_open_close, btc_daily_high_low, on='date')
btc_daily['date'] = pd.to_datetime(btc_daily['date'])

In [21]:
btc_daily_tweets = pd.merge(bitcoin_tweets_df_day_cleaned.reset_index(), bitcoin_tweets_df_day_agg_cleaned.reset_index(), on='date', suffixes=('_overall', '_aggregated'))

In [24]:
btc_daily_data=pd.merge(btc_daily, btc_daily_tweets, on='date').sort_values(by='date')

In [25]:
btc_hourly_open_close = btc_price_data[['time','hour']].groupby(['hour']).time.agg([min,max]).reset_index()
btc_hourly_high=btc_price_data.groupby(['hour']).high.agg(max).reset_index()
btc_hourly_low=btc_price_data.groupby(['hour']).low.agg(min).reset_index()
btc_hourly_open_close.columns=['hour','open_time','close_time']
btc_hourly_open = pd.merge(btc_hourly_open_close, btc_price_data, left_on='open_time', right_on='time')[['hour_x','open','volume']]
btc_hourly_close = pd.merge(btc_hourly_open_close, btc_price_data, left_on='close_time', right_on='time')[['hour_x','close','volume']]
btc_hourly_high_low = pd.merge(btc_hourly_high, btc_hourly_low, on='hour')
btc_hourly_open_close = pd.merge(btc_hourly_open, btc_hourly_close, on='hour_x')
btc_hourly_open_close.columns=['hour','open','open_volume','close','close_volume']
btc_hourly=pd.merge(btc_hourly_open_close, btc_hourly_high_low, on='hour')
btc_hourly['hour'] = pd.to_datetime(btc_hourly['hour'])

In [26]:
btc_hourly_tweets = pd.merge(bitcoin_tweets_df_hour_cleaned.reset_index(), bitcoin_tweets_df_hour_agg_cleaned.reset_index(), on='date', suffixes=('_overall', '_aggregated'))

In [27]:
btc_hourly.columns=['date', 'open', 'open_volume', 'close', 'close_volume', 'high', 'low']

In [28]:
btc_hourly_data=pd.merge(btc_hourly, btc_hourly_tweets, on='date').sort_values(by='date')

In [29]:
btc_daily_data.to_csv('Data/bitcoin_daily_price_sent.csv', header=True, index=False)
btc_hourly_data.to_csv('Data/bitcoin_hourly_price_sent.csv', header=True, index=False)